In [39]:
# -*- coding: utf-8 -*-
# this script is responsible for training the neural networks

from scipy import io
import pandas as pd
import numpy as np
import time
import pickle
# import os
import h5py
import sys, getopt
import datetime
from MapCallback import MapCallback

In [40]:
# 另一個常見的寫法是
import os
os.environ['CUDA_VISIBLE_DEVICES']='0,1'

In [41]:
nb_epochs = 2 # number of epochs, should be high, the end of the learning process is controled by early stoping
es_patience = 100 # patience for early stoping 
batchSize = 10 # batch size for mini-batch training
hdf5path = '../birdclef_data/data_top468_nozero.hdf5' # training data generated by loadData.py
# modelPath = './model-AlexNet.py' # filename of the model to use (currently model-birdClef.py or model-AlexNet.py)
modelPath = 'InceptionV3'
logfileName = 'log.xls'
#scalerFilePath = '../birdclef_data/standardScaler_5000.pickle'
scalerFilePath = None
preTrainedModelWeightsPath = None # path and filename to pretrained network: if there is a pretrained network, we can load it and continue to train it
tensorflowBackend = True # set true if Keras has TensorFlow backend - this way we set TF not to allocate all the GPU memory

if (tensorflowBackend):
    import tensorflow as tf
    config = tf.ConfigProto()
    config.gpu_options.allow_growth=True
    sess = tf.Session(config=config)
    from keras import backend as K
    K.set_session(sess)

print('nb_epochs: %d, hdf5path: %s, scalerFilePath: %s' % (nb_epochs, hdf5path,scalerFilePath))

nb_epochs: 2, hdf5path: ../birdclef_data/data_top468_nozero.hdf5, scalerFilePath: None


In [42]:
scaler = None
scaleData = None
# if a scaler file generated by loadData.py is given, than load it and define a scaler function that will be used later
if scalerFilePath is not None:
    scaler = pickle.load(open(scalerFilePath, 'rb'))
    # Can't use scaler.transform because it only supports 2d arrays.
    def scaleData(X):
        return (X-scaler.mean_)/scaler.scale_

In [43]:
from io_utils_mod import HDF5Matrix
f = h5py.File(hdf5path, 'r')
X = f.get('X')
y = f.get('y')
print("Shape of X: ")
print(X.shape)
dataSetLength = X.shape[0]
output_dim = y.shape[1] #len(y_train[0])
# test and validation splits
testSplit = 0.01 # 1%
validationSplit	= 0.05 #####0.05 # 5%
f.close()

Shape of X: 
(5530, 1, 200, 310)


In [44]:
print('dataSetLength={}'.format(dataSetLength))
print('output_dim={}'.format(output_dim))

dataSetLength=5530
output_dim=468


In [45]:
# load training data
X_train = HDF5Matrix(hdf5path, 'X', 0, int(dataSetLength*(1-(testSplit+validationSplit))), normalizer = scaleData) 
y_train = HDF5Matrix(hdf5path, 'y', 0, int(dataSetLength*(1-(testSplit+validationSplit))))
# load validation data
X_validation = HDF5Matrix(hdf5path, 'X', int(dataSetLength*(1-(testSplit+validationSplit)))+1, int(dataSetLength*(1-testSplit)), normalizer = scaleData)
y_validation = HDF5Matrix(hdf5path, 'y', int(dataSetLength*(1-(testSplit+validationSplit)))+1, int(dataSetLength*(1-testSplit)))
# load test data
X_test = HDF5Matrix(hdf5path, 'X', int(dataSetLength*(1-testSplit))+1, dataSetLength, normalizer = scaleData)
y_test = HDF5Matrix(hdf5path, 'y', int(dataSetLength*(1-testSplit))+1, dataSetLength)

print("Shape of X_train after train-validation-test split:")
print('   X_train, y_train=', X_train.shape, y_train.shape, type(X_train), type(y_train))
print('   X_validation, y_validation=', X_validation.shape, y_validation.shape)
print('   X_test, y_test=', X_test.shape, y_test.shape)

Shape of X_train after train-validation-test split:
   X_train, y_train= (5198, 1, 200, 310) (5198, 468) <class 'io_utils_mod.HDF5Matrix'> <class 'io_utils_mod.HDF5Matrix'>
   X_validation, y_validation= (275, 1, 200, 310) (275, 468)
   X_test, y_test= (55, 1, 200, 310) (55, 468)


### vvvvv 將 AlexNet 換成 InceptionV3 Model vvvvv

In [46]:
from keras.applications.inception_v3 import InceptionV3
from keras.callbacks import TensorBoard, ModelCheckpoint, LearningRateScheduler
from keras.models import Model
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
# import keras.backend as K

# def preprocess_input(x):
#     x /= 255.
#     x -= 0.5
#     x *= 2.
#     return x

In [47]:
# lr decay schedule
def lr_schedule(epoch):
    """Learning Rate Schedule
    Learning rate is scheduled to be reduced after 80, 120epochs.
    Called automatically every epoch as part of callbacks during training.
    # Arguments
        epoch (int): The number of epochs
    # Returns
        lr (float32): learning rate
    """
    lr = 1e-4
    decay = int(epoch / 10)
    if decay != 0:
        lr /= (10 ** decay)
    print('Learning rate = {}, decay = {}'.format(lr, decay))
    return lr

In [48]:
K.set_image_data_format('channels_first')
# from keras.utils.training_utils import multi_gpu_model
# gpu_count = 2

model = InceptionV3(include_top=False, weights=None, input_shape=(1,200,310), pooling='avg')
# model = InceptionV3(include_top=False, weights=None, input_shape=(1,200,310), pooling='avg')

# parallel_model = multi_gpu_model(model, gpus=gpu_count)

input_tensor = model.input
# input_tensor = get_input_at(node_index)
# build top
x = model.output
x = Dropout(.5)(x)
x = Dense(output_dim, activation='softmax')(x)

model = Model(inputs=input_tensor, outputs=x)

for layer in model.layers:
    layer.W_regularizer = l2(1e-2)
    layer.trainable = True

# call backs
if os.path.exists('./modelWeights/') is False:
    os.mkdir('./modelWeights/') #M:
checkpointer = ModelCheckpoint(filepath='./modelWeights/weights.h5', verbose=1, save_best_only=True)
lr = LearningRateScheduler(lr_schedule)
model.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])
# parallel_model.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])
# batch_size = batchSize * gpu_count

In [49]:
# 如果大家的server有開多GPU 且用keras的話，可以用下面的multi GPU codes 加速...
# from keras.utils.training_utils import multi_gpu_model
# gpu_count = 2
# model = InceptionV3(include_top=False, weights=None, input_shape=(1,200,310), pooling='avg')
# parallel_model = multi_gpu_model(model, gpus=gpu_count)
# parallel_model.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])
# batch_size = batchSize * gpu_count
# ...

In [50]:
# model.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])

In [51]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 1, 200, 310)  0                                            
__________________________________________________________________________________________________
conv2d_189 (Conv2D)             (None, 32, 99, 154)  288         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_189 (BatchN (None, 32, 99, 154)  96          conv2d_189[0][0]                 
__________________________________________________________________________________________________
activation_189 (Activation)     (None, 32, 99, 154)  0           batch_normalization_189[0][0]    
__________________________________________________________________________________________________
conv2d_190

### ^^^^^ 將 AlexNet 換成 InceptionV3 Model ^^^^^

In [52]:
# load model and compile it, we use RMSprop here, other optimizer algorithm should be tested
# execfile(modelPath)
# model.compile(loss='categorical_crossentropy', optimizer='rmsprop')#, metrics=["accuracy"])

# print the model
# print("The following model is used: ")
# for layer in model.layers:
#     print("{} output shape: {}".format(layer.name, layer.output_shape))

# load pretrained model if it is set
# if preTrainedModelWeightsPath is not None:
#     model.load_weights(preTrainedModelWeightsPath)
#     print("Reloaded weights from: {}".format(preTrainedModelWeightsPath))

# define callback functions
mapcallback	= MapCallback()

earlyStopping = EarlyStopping(monitor='val_loss', patience = es_patience) # early stoping

# save best models based on accuracy, loss and MAP metrics
#bestModelFilePath_val_map = './modelWeights/best_val_map_{}_{}.hdf5'.format(output_dim, datetime.datetime.now().strftime('%Y-%m-%d-%M-%S'))
#bestModelFilePath_val_acc = './modelWeights/best_val_acc_{}_{}.hdf5'.format(output_dim, datetime.datetime.now().strftime('%Y-%m-%d-%M-%S'))
#bestModelFilePath_val_loss = './modelWeights/best_val_loss_{}_{}.hdf5'.format(output_dim, datetime.datetime.now().strftime('%Y-%m-%d-%M-%S'))


# bestModelFilePath_val_acc = './modelWeights/best_val_acc_{}.hdf5'.format(output_dim)
# bestModelFilePath_val_loss = './modelWeights/best_val_loss_{}.hdf5'.format(output_dim)
# bestModelFilePath_val_map = './modelWeights/best_val_map_{}.hdf5'.format(output_dim)
# checkpointer_val_acc = ModelCheckpoint(filepath = bestModelFilePath_val_acc, verbose = 1, monitor = 'val_acc', save_best_only = True)
# checkpointer_val_loss = ModelCheckpoint(filepath = bestModelFilePath_val_loss, verbose = 1, monitor = 'val_loss', save_best_only = True)
# checkpointer_val_map = ModelCheckpoint(filepath = bestModelFilePath_val_map, verbose = 1, monitor = 'val_map', mode = 'max', save_best_only = True)

In [53]:
# training
#M: convert HDF5 array to Numpy array
X_train = np.array(X_train)
y_train = np.array(y_train)
X_validation = np.array(X_validation)
y_validation = np.array(y_validation)

In [54]:
print(X_validation.shape, y_validation.shape)
# y_validation[1]

(275, 1, 200, 310) (275, 468)


In [55]:
# store the starting time 
startTime = time.time()

# fitting_result = model.fit(X_train, y_train, epochs = nb_epochs, batch_size = batchSize, callbacks = [earlyStopping, mapcallback, checkpointer_val_acc, checkpointer_val_loss,  checkpointer_val_map], shuffle = 'batch', validation_data = (X_validation, y_validation))
fitting_result = model.fit(X_train, y_train, epochs = nb_epochs, batch_size = batchSize, 
                           callbacks = [earlyStopping, checkpointer, lr], 
#                            callbacks = [checkpointer, lr], 
                           shuffle = 'batch', 
                           validation_data = (X_validation, y_validation))

# calculate the elapsed time
elapsed = time.time()-startTime;
print("Execution time: {0} s".format(elapsed))

# # save model
# model.save('trainModel_InceptionV3.h5')



Train on 5198 samples, validate on 275 samples
Learning rate = 0.0001, decay = 0
Epoch 1/2
5190/5198 [============================>.] - ETA: 0s - loss: 6.5757 - acc: 0.0104
Epoch 00001: val_loss improved from inf to 6.83753, saving model to ./modelWeights/weights.h5
5198/5198 [==============================] - 132s 25ms/step - loss: 6.5734 - acc: 0.0104 - val_loss: 6.8375 - val_acc: 0.0000e+00
Learning rate = 0.0001, decay = 0
Epoch 2/2
5190/5198 [============================>.] - ETA: 0s - loss: 6.0531 - acc: 0.0202
Epoch 00002: val_loss did not improve
5198/5198 [==============================] - 57s 11ms/step - loss: 6.0516 - acc: 0.0202 - val_loss: 9.4134 - val_acc: 0.0000e+00
Execution time: 201.37748193740845 s


In [56]:
# save model
model.save('trainModel_InceptionV3.h5')

In [57]:
# convert the output (probabilistics) to classes
def proba_to_class(a):
    classCount = len(a[0])
    to_return = np.empty((0,classCount))
    for row in a:
        maxind = np.argmax(row)
        to_return = np.vstack((to_return, [1 if i == maxind else 0 for i in range(classCount)]))
    return to_return

In [58]:
# calculate metrics on test data with the last model 
from sklearn.metrics import average_precision_score, accuracy_score
y_result = model.predict(X_test)
map = average_precision_score( y_test.data[y_test.start: y_test.end], y_result, average='micro')
accuracy = accuracy_score(y_test.data[y_test.start: y_test.end], proba_to_class(y_result))
print("AveragePrecision: {}".format(map))
print("Accuracy: {}".format(accuracy))

AttributeError: 'HDF5Matrix' object has no attribute 'ndim'

In [ ]:
# reload the best model with smallest validation loss and calculate metrics on test data
print("----- Loading best model from: {}  -------".format(bestModelFilePath_val_loss))
model.load_weights(bestModelFilePath_val_loss)
y_result_bm = model.predict(X_test)
map_bm_val_loss = average_precision_score( y_test.data[y_test.start: y_test.end], y_result_bm, average='macro')
accuracy_bm_val_loss = accuracy_score(y_test.data[y_test.start: y_test.end], proba_to_class(y_result_bm))
print("AveragePrecision: {}".format(map_bm_val_loss))
print("Accuracy: {}".format(accuracy_bm_val_loss))

In [ ]:
# reload the best model with highest validation accuracy and calculate metrics on test data
print("----- Loading best model from: {}  -------".format(bestModelFilePath_val_acc))
model.load_weights(bestModelFilePath_val_acc)
y_result_bm = model.predict(X_test)
map_bm_val_acc = average_precision_score( y_test.data[y_test.start: y_test.end], y_result_bm, average='macro')
accuracy_bm_val_acc = accuracy_score(y_test.data[y_test.start: y_test.end], proba_to_class(y_result_bm))
print("AveragePrecision: {}".format(map_bm_val_acc))
print("Accuracy: {}".format(accuracy_bm_val_acc))

In [ ]:
# save the results summery into an excel file
import log
log.logToXLS(logfileName, model, fitting_result, {'execution(s)':elapsed, 'map':map, 'accuracy':accuracy, 'map_bm_val_loss':map_bm_val_loss, 'accuracy_bm_val_loss':accuracy_bm_val_loss,'map_bm_val_acc':map_bm_val_acc, 'accuracy_bm_val_acc':accuracy_bm_val_acc, 'modelPyFile': modelPath})

In [ ]:
# 在準備好 X_train ....後加一個 cell

import matplotlib.pyplot as plt
%matplotlib inline
i=3
img = X_train[i].reshape((200,310))
print(X_train[i].shape, img.shape)
plt.imshow(img, interpolation='nearest')